# Introduction to [PowerSystems.jl](https://github.com/NREL-SIIP/PowerSystems.jl)

**Originally Contributed by**: Clayton Barrows and Jose Daniel Lara

## Introduction

This notebook is intended to show a power system data specification framework that exploits the
capabilities of Julia to improve performance and allow modelers to develop modular software
to create problems with different complexities and enable large scale analysis.

### Objective
PowerSystems.jl provides a type specification for bulk power system data.
The objective is to exploit Julia's integration of dynamic types to enable efficient data
handling and enable functional dispatch in modeling and analysis applications
As explained in Julia's documentation:

"Julia’s type system is dynamic, but gains some of the advantages of static type systems
by making it possible to indicate that certain values are of specific types. This can be
of great assistance in generating efficient code, but even more significantly, it allows
method dispatch on the types of function arguments to be deeply integrated with the language."

For more details on Julia types, refer to the [documentation](https://docs.julialang.org/en/v1/)


## Environment and packages

PowerSystems.jl relies on a framework for data handling established in
[InfrastructureSystems.jl](https://github.com/NREL-SIIP/InfrastructureSystems.jl).
Users of PowerSystems.jl should not need to interact directly with InfrastructureSystems.jl

The examples in this notebook depend upon Julia 1.2 and a specific set of package releases
as defined in the `Manifest.toml`.

In [1]:
using Pkg
Pkg.status()

using SIIPExamples;
using PowerSystems;
using D3TypeTrees;

Project SIIPExamples v0.0.1
Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [336ed68f] CSV v0.7.4
  [9961bab8] Cbc v0.7.0
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.21.4
  [becb17da] Feather v0.5.6
  [2cd47ed4] InfrastructureSystems v0.13.0 [`../InfrastructureSystems.jl`]
  [b6b21f68] Ipopt v0.6.2
  [2535ab7d] JSON2 v0.3.2
  [4076af6c] JuMP v0.21.3
  [98b081ad] Literate v2.5.0
  [47be7bcc] ORCA v0.4.0
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v1.5.5
  [5f7eddb3] PowerGraphics v0.4.2 [`../PowerGraphics.jl`]
  [e690365d] PowerSimulations v0.6.0 [`../PowerSimulations.jl`]
  [bcd98974] PowerSystems v0.23.2 [`../PowerSystems.jl`]
  [9e3dc215] TimeSeries v0.18.0
  [f269a46b] TimeZones v1.3.1
  [0f1e0344] WebIO v0.8.14
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [44cfe95a] Pkg 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 


## Types in PowerSystems
PowerSystems.jl provides a type hierarchy for specifying power system data. Data that
describes infrastructure components is held in `struct`s. For example, a `Bus` is defined
as follows with fields for the parameters required to describe a bus (along with an
`internal` field used by InfrastructureSystems to improve the efficiency of handling data).

In [2]:
print_struct(Bus)

mutable struct PowerSystems.Bus
    number::Int64
    name::String
    bustype::Union{Nothing, PowerSystems.BusTypes.BusType}
    angle::Union{Nothing, Float64}
    magnitude::Union{Nothing, Float64}
    voltage_limits::Union{Nothing, NamedTuple{(:min, :max),Tuple{Float64,Float64}}}
    base_voltage::Union{Nothing, Float64}
    area::Union{Nothing, PowerSystems.Area}
    load_zone::Union{Nothing, PowerSystems.LoadZone}
    ext::Dict{String,Any}
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


### Type Hierarchy
PowerSystems is intended to organize data containers by the behavior of the devices that
the data represents. To that end, a type hierarchy has been defined with several levels of
abstract types starting with `PowerSystemType`:
- `Component`: includes all elements of power system data
  - `PowerSystems.Topology`: includes non physical elements describing network connectivity
  - `Service`: includes descriptions of system requirements (other than energy balance)
  - `Device`: includes descriptions of all the physical devices in a power system
- `PowerSystems.TechnicalParams`: includes structs that hold data describing the technical or economic capabilities of `Device`some
- `System`: collects all of the `Component`s

*The following trees are made with [D3TypeTrees](https://github.com/claytonpbarrows/D3TypeTrees.jl),
nodes that represent Structs will show the Fields in the hoverover tooltip.*

In [3]:
TypeTree(PowerSystemType)

PowerSystems.PowerSystemType
├──PowerSystems.Component
│  ├──PowerSystems.Service
│  │  ├──PowerSystems.AGC (0 children)
│  │  ├──PowerSystems.Reserve (3 children)
│  │  └──PowerSystems.Transfer (0 children)
│  ├──PowerSystems.Device
│  │  ├──PowerSystems.RegulationDevice (0 children)
│  │  ├──PowerSystems.Branch (2 children)
│  │  ├──PowerSystems.StaticInjection (4 children)
│  │  └──PowerSystems.DynamicInjection (2 children)
│  └──PowerSystems.Topology
│     ├──PowerSystems.Bus (0 children)
│     ├──PowerSystems.AggregationTopology (2 children)
│     └──PowerSystems.Arc (0 children)
├──PowerSystems.System
└──PowerSystems.DeviceParameter
   ├──PowerSystems.DynamicComponent
   │  ├──PowerSystems.DynamicInverterComponent (6 children)
   │  ├──PowerSystems.DynamicGeneratorComponent (5 children)
   │  └──PowerSystems.InverterComponent (0 children)
   ├──PowerSystems.OperationalCost
   │  ├──PowerSystems.MultiStartCost (0 children)
   │  ├──PowerSystems.TwoPartCost (0 children)
   │  └──PowerSystems.ThreePartCost (0 children)
   ├──PowerSystems.ActivePowerControl
   │  └──PowerSystems.VirtualInertia (0 children)
   └──PowerSystems.ReactivePowerControl
      └──PowerSystems.ReactivePowerDroop (0 children)

### `Forecasts`
Every `Component` has a `_forecasts::InfrastructureSystems.Forecasts` field (even composite types).
`Forecasts` are used to hold time series information that describes the temporally dependent
data of fields within the same struct. For example, the `ThermalStandard._forecasts` field can
describe other fields in the struct (`available`, `activepower`, `reactivepower`), while
the `ThermalStandard.tech._forecasts` holds data for `rating` and other `TechThermal` fields.

`Forecast`s themselves can take the form of the following:

In [4]:
TypeTree(Forecast)

InfrastructureSystems.Forecast
├──InfrastructureSystems.ScenarioBased
├──InfrastructureSystems.PiecewiseFunction
├──InfrastructureSystems.Probabilistic
└──InfrastructureSystems.Deterministic

In each case, the forecast contains fields for `label` and `data` to identify the `Component`
field that the forecast describes, and the time series `data`. For example:

In [5]:
print_struct(Deterministic)

mutable struct InfrastructureSystems.Deterministic
    label::String
    data::TimeSeries.TimeArray
end


Examples of how to create and add forecasts to system can be found in the
[Add Forecasts Example](../PowerSystems.jl Examples/add_forecasts.ipynb)

### System
The `System` object collects all of the individual components into a single struct along
with some metadata about the system itself (e.g. `basepower`)

In [6]:
print_struct(System)

 struct PowerSystems.System
    data::InfrastructureSystems.SystemData
    frequency::Float64
    bus_numbers::Set{Int64}
    runchecks::Bool
    units_settings::InfrastructureSystems.SystemUnitsSettings
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


## Basic example
PowerSystems contains a few basic data files (mostly for testing and demonstration).

In [7]:
BASE_DIR = abspath(joinpath(dirname(Base.find_package("PowerSystems")), ".."))
include(joinpath(BASE_DIR, "test", "data_5bus_pu.jl")) #.jl file containing 5-bus system data
nodes_5 = nodes5() # function to create 5-bus buses

5-element Array{PowerSystems.Bus,1}
nodeA (PowerSystems.Bus)
nodeB (PowerSystems.Bus)
nodeC (PowerSystems.Bus)
nodeD (PowerSystems.Bus)
nodeE (PowerSystems.Bus)


### Create a `System`

In [8]:
sys = System(
    nodes_5,
    vcat(thermal_generators5(nodes_5), renewable_generators5(nodes_5)),
    loads5(nodes_5),
    branches5(nodes_5),
    nothing,
    100.0,
    nothing,
    nothing,
)

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5
2,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,6
3,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
4,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
5,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,5


### Accessing `System` Data
PowerSystems provides functional interfaces to all data. The following examples outline
the intended approach to accessing data expressed using PowerSystems.

PowerSystems enforces unique `name` fields between components of a particular concrete type.
So, in order to retrieve a specific component, the user must specify the type of the component
along with the name and system

#### Accessing components

In [9]:
@show get_component(Bus, sys, "nodeA")
@show get_component(Line, sys, "1")

get_component(Bus, sys, "nodeA") = PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("af2b975e-5fd1-45c1-bf2e-db0d3aea4950"), InfrastructureSystems.SystemUnitsSettings(100.0, InfrastructureSystems.SYSTEM_BASE), nothing))
get_component(Line, sys, "1") = PowerSystems.Line("1", true, 0.0, 0.0, PowerSystems.Arc(PowerSystems.Bus(1, "nodeA", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("af2b975e-5fd1-45c1-bf2e-db0d3aea4950"), InfrastructureSystems.SystemUnitsSettings(100.0, InfrastructureSystems.SYSTEM_BASE), nothing)), PowerSystems.Bus(2, "nodeB", PowerSystems.BusTypes.PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("36ce1b23-44a1-46b0-8f5a-5de

1 (PowerSystems.Line):
   name: 1
   available: true
   active_power_flow: 0.0
   reactive_power_flow: 0.0
   arc: nodeA -> nodeB: (PowerSystems.Arc)
   r: 0.00281
   x: 0.0281
   b: (from = 0.00356, to = 0.00356)
   rate: 2.0
   angle_limits: (min = -0.7, max = 0.7)
   services: 0-element Array{PowerSystems.Service,1}
   ext: Dict{String,Any}()
   forecasts: InfrastructureSystems.Forecasts: 0

Similarly, you can access all the components of a particular type: *note: the return type
of get_components is a `FlattenIteratorWrapper`, so call `collect` to get an `Array`

In [10]:
get_components(Bus, sys) |> collect

5-element Array{PowerSystems.Bus,1}
nodeA (PowerSystems.Bus)
nodeC (PowerSystems.Bus)
nodeE (PowerSystems.Bus)
nodeB (PowerSystems.Bus)
nodeD (PowerSystems.Bus)


`get_components` also works on abstract types:

In [11]:
get_components(Branch, sys) |> collect

6-element Array{PowerSystems.Branch,1}
4 (PowerSystems.Line)
1 (PowerSystems.Line)
5 (PowerSystems.Line)
2 (PowerSystems.Line)
6 (PowerSystems.Line)
3 (PowerSystems.Line)


The fields within a component can be accessed using the `get_*` functions:

In [12]:
bus1 = get_component(Bus, sys, "nodeA")
@show get_name(bus1);
@show get_magnitude(bus1);

get_name(bus1) = "nodeA"
get_magnitude(bus1) = 1.0


#### Accessing `Forecast`s

First we need to add some forecasts to the `System`

In [13]:
loads = collect(get_components(PowerLoad, sys))
for (l, ts) in zip(loads, load_timeseries_DA[2])
    add_forecast!(sys, l, Deterministic("activepower", ts))
end

Now that the system has some forecasts, we can see all of them:

In [14]:
iterate_forecasts(sys) |> collect

3-element Array{Any,1}:
 InfrastructureSystems.Deterministic("activepower", 24×1 TimeSeries.TimeArray{Float64,1,Dates.DateTime,Array{Float64,1}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00)
 InfrastructureSystems.Deterministic("activepower", 24×1 TimeSeries.TimeArray{Float64,1,Dates.DateTime,Array{Float64,1}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00)
 InfrastructureSystems.Deterministic("activepower", 24×1 TimeSeries.TimeArray{Float64,1,Dates.DateTime,Array{Float64,1}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00)

If we want to access a specific forecast for a specific component, we need to specify:
 - Forecast type
 - `component`
 - initial_time
 - label

We can find the unique set of initial times of all the forecasts in the system:

In [15]:
get_forecast_initial_times(sys)

1-element Array{Dates.DateTime,1}:
 2024-01-02T00:00:00

Or for a specific component:

In [16]:
@show initial_times = get_forecast_initial_times(Deterministic, loads[1]);

initial_times = get_forecast_initial_times(Deterministic, loads[1]) = Dates.DateTime[2024-01-02T00:00:00]


We can find the fields for which a component has a forecast:

In [17]:
@show labels = get_forecast_labels(Deterministic, loads[1], initial_times[1])

labels = get_forecast_labels(Deterministic, loads[1], initial_times[1]) = ["activepower"]


1-element Array{String,1}:
 "activepower"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*